# --- Pickle Pulse Generator ---

## Import libraries and configure board as a remote object

In [31]:
# Import libraries
from qick import *
from pickle_parser import *
import Pyro4
%pylab inline

# Pyro communicates with board
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

# Static IP proxy
ns_host = "192.168.2.99"
ns_port = 8888
proxy_name = "myqick"

# QICK object
ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib

QICK configuration:

	Board: RFSoC4x2

	Software version: 0.2.267
	Firmware timestamp: Wed Sep  6 18:49:29 2023

	Global clocks (MHz): tProcessor 409.600, RF reference 491.520

	2 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 0, blk 0 is DAC_B
	1:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 2, blk 0 is DAC_A

	2 readout channels:
	0:	axis_readout_v2 - configured by PYNQ
		fs=4423.680 MHz, decimated=552.960 MHz, 32-bit DDS, range=4423.680 MHz
		maxlen 16384 accumulated, 1024 decimated (1.852 us)
		triggered by output 7, pin 14, feedback to tProc input 0
		ADC tile 0, blk 0 is ADC_D
	1:	axis_readout_v2 - configured by PYNQ


## Import and parse pickle file containing sequences of pulses

In [32]:
# Import dictionary of sequences from pickle file
import pickle
with open('Sequence_ch1_ch2_mw.pickle', 'rb') as handle:
    raman_cooling_seqs = pickle.load(handle)

# pulse_seqs = {}
# pulse_seqs["DAC_A"] = raman_cooling_seqs["MW"]
# pulse_seqs["PMOD_0"] = raman_cooling_seqs["ch2_Dig"]
# pulse_seqs["PMOD_1"] = raman_cooling_seqs["ch1_Dig"]

# Method 1: use if keys don't match channel names ------------------------------
ch_map={"DAC_A":"MW", "PMOD_0":"ch2_Dig", "PMOD_1":"ch1_Dig"}
raman_cooling = PickleParse(raman_cooling_seqs, ch_map)

# Method 2: use if channel names already configured (uncomment block above) ----
# raman_cooling = PickleParse(pulse_seqs)

----- DAC -----
pulse_lens [47.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
times [0.1, 50.65, 54.65, 58.65, 62.65, 66.65, 70.65, 74.65, 78.65, 82.65, 86.65, 90.65]
freqs [50.0, 50.0, 55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90.0, 95.0, 100.0]
finish 94.3
num_pulses 12
ch_type DAC
ch_ref 1
----- PMOD ------
----- PMOD ------
Last finish: 94.3


## Initialise program and load default parameters

In [33]:
prog = QickProgram(soccfg) # Create program object
raman_cooling.dac_defaults(prog, gain=10000, freq=50, phase=0) # Necessary step!

# Method 1 - Run, specifying number of repetitions -----------------------------
raman_cooling.generate_asm(prog, reps=10)

# Method 2 - Run with no repetitions -------------------------------------------
# raman_cooling.generate_asm(prog)

# Run program and print QICK assembly

In [34]:
prog.config_all(soc) # Load assembly to board
soc.tproc.start() # Run program
print(prog) # Useful for debugging synchronisation issues


// Program

        regwi 1, $23, 0;                        //phase = 0
        regwi 1, $25, 10000;                    //gain = 10000
        synci 200;
        regwi 0, $14, 9;
LOOP_I: regwi 1, $22, 21845333;                 //freq = 21845333
        regwi 1, $26, 618701;                   //phrst| stdysel | mode | | outsel = 0b01001 | length = 28877 
        regwi 1, $27, 3;                        //t = 3
        set 1, 1, $22, $23, $0, $25, $26, $27;  //ch = 1, pulse @t = $27
        regwi 1, $22, 21845333;                 //freq = 21845333
        regwi 1, $26, 589885;                   //phrst| stdysel | mode | | outsel = 0b01001 | length = 61 
        regwi 1, $27, 20708;                    //t = 20708
        set 1, 1, $22, $23, $0, $25, $26, $27;  //ch = 1, pulse @t = $27
        regwi 1, $22, 24029867;                 //freq = 24029867
        regwi 1, $26, 589885;                   //phrst| stdysel | mode | | outsel = 0b01001 | length = 61 
        regwi 1, $27, 22347;     